### --- Day 3: Gear Ratios ---
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```
In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [102]:
test_data = [
    "467..114..",
    "...*......",
    "..35..633.",
    "......#...",
    "617*......",
    ".....+.58.",
    "..592.....",
    "......755.",
    "...$.*....",
    ".664.598..",    
]
test_data = [list(row) for row in test_data]
result = 4361

In [103]:
from dataclasses import dataclass


@dataclass
class NumberPos:
    row: int
    col: int
    length: int
    value: int


@dataclass
class SymbolPos:
    row: int
    col: int
    symbol: str


def find_all_unique_symbols(data: list[list[str]]) -> set[str]:
    return {
        symbol
        for row in data
        for symbol in row
        if symbol != "." and not symbol.isdigit()
    }


def find_all_symbol_positions(
    data: list[list[str]], symbols: list[str]
) -> list[SymbolPos]:
    return [
        SymbolPos(row, col, data[row][col])
        for row in range(len(data))
        for col in range(len(data[row]))
        if data[row][col] in symbols
    ]


def find_all_number_postions(data: list[list[str]]) -> list[NumberPos]:
    numbers = []
    for i, line in enumerate(data):
        j = 0
        while j < len(line):
            if line[j].isdigit():
                length = 1
                while j + length < len(line) and line[j + length].isdigit():
                    length += 1
                numbers.append(
                    NumberPos(i, j, length, int("".join(line[j : j + length])))
                )
                j += length
            else:
                j += 1
    return numbers


def is_symbol_and_number_adjecent(symbol_pos: SymbolPos, number_pos: NumberPos) -> bool:
    acceptable_col_values = [
        col_number
        for col_number in range(
            number_pos.col - 1, number_pos.col + number_pos.length + 1
        )
    ]
    return (
        abs(symbol_pos.row - number_pos.row) <= 1
        and symbol_pos.col in acceptable_col_values
    )


def total_count(numbers_pos: list[NumberPos], symbols_pos: list[SymbolPos]) -> int:
    total_value = 0
    for number_pos in numbers_pos:
        for symbol_pos in symbols_pos:
            if is_symbol_and_number_adjecent(symbol_pos, number_pos):
                total_value += number_pos.value
                break
    return total_value

In [104]:
# test
symbols = find_all_unique_symbols(test_data)
symbol_positions = find_all_symbol_positions(test_data, symbols)
number_positions = find_all_number_postions(test_data)
print(
    f"Expected result: {result}, actual result: {total_count(number_positions, symbol_positions)}"
)

Expected result: 4361, actual result: 4361


In [105]:
with open("data/day03.txt") as f:
    data = f.read().splitlines()
data = [list(row) for row in data]

symbols = find_all_unique_symbols(data)
symbol_positions = find_all_symbol_positions(data, symbols)
number_positions = find_all_number_postions(data)
print(total_count(number_positions, symbol_positions))

551094


### --- Part Two ---
The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```
In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

In [106]:
result2 = 467835

def gear_ratio_count(numbers_pos: list[NumberPos], symbols_pos: list[SymbolPos]) -> int:
    total_value = 0
    gear_symbols = [symbol for symbol in symbols_pos if symbol.symbol == "*"]
    for gear_symbol in gear_symbols:
        gear_ratio = 1
        adjacent_numbers = 0
        # if number of adjacent numbers is exactly 2, then multiply these numbers to get the gear ratio and move to another gear symbol
        for number_pos in numbers_pos:
            if is_symbol_and_number_adjecent(gear_symbol, number_pos):
                gear_ratio *= number_pos.value
                adjacent_numbers += 1
            if adjacent_numbers == 2:
                total_value += gear_ratio
                break

    return total_value

# test
symbols = find_all_unique_symbols(test_data)
symbol_positions = find_all_symbol_positions(test_data, symbols)
number_positions = find_all_number_postions(test_data)
print(
    f"Expected result: {result2}, actual result: {gear_ratio_count(number_positions, symbol_positions)}"
)
    

Expected result: 467835, actual result: 467835


In [107]:
with open("data/day03.txt") as f:
    data = f.read().splitlines()

data = [list(row) for row in data]

symbols = find_all_unique_symbols(data)
symbol_positions = find_all_symbol_positions(data, symbols)
number_positions = find_all_number_postions(data)
print(gear_ratio_count(number_positions, symbol_positions))

80179647
